In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Importação da bibliotecas usadas;**

In [ ]:
from category_encoders import *
from category_encoders import CountEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer


**Pegando os dados a serem utilizados;**

In [ ]:
test_original = pd.read_csv('/kaggle/input/mini-flight-delay-prediction/flight_delays_test.csv')
train_original = pd.read_csv('/kaggle/input/mini-flight-delay-prediction/flight_delays_train.csv')
test_mod = test_original.copy()
train_mod = train_original.copy()

In [ ]:
test_mod.head()

In [ ]:
train_mod.head()

**Tratando os dados;**

In [ ]:
def transformar_valor(valor):
    if valor == 'Y':
        return 1
    else:
        return 0


train_mod['dep_delayed_15min'] = train_mod['dep_delayed_15min'].map(transformar_valor)

train_mod['DayOfWeek'] = train_mod['DayOfWeek'].str.replace('\w-', '')
train_mod['DayofMonth'] = train_mod['DayofMonth'].str.replace('\w-', '')
train_mod['Month'] = train_mod['Month'].str.replace('\w-', '')

test_mod['DayOfWeek'] = test_mod['DayOfWeek'].str.replace('\w-', '')
test_mod['DayofMonth'] = test_mod['DayofMonth'].str.replace('\w-', '')
test_mod['Month'] = test_mod['Month'].str.replace('\w-', '')

label = train_mod['dep_delayed_15min']
variaveis = ['Month','DayofMonth','DayOfWeek','DepTime','UniqueCarrier','Origin','Dest','Distance']
df = pd.concat([train_mod[variaveis],test_mod])

df = pd.concat([train_mod[variaveis],test_mod])
dest_y_train = train_mod['Dest']
uc_y_train = train_mod['UniqueCarrier']
enc_aerporto = CountEncoder(cols=['Dest', 'Origin']).fit(df, dest_y_train)
enc_uc = CountEncoder(cols=['UniqueCarrier']).fit(df, uc_y_train)
train_mod = enc_aerporto.transform(train_mod[variaveis])
train_mod = enc_uc.transform(train_mod[variaveis])

test_mod = enc_aerporto.transform(test_mod)
test_mod = enc_uc.transform(test_mod)

In [ ]:
test_mod.head()

In [ ]:
train_mod.head()

**Criando o modelo e treinado-o;**

In [ ]:
modelo = MLPClassifier(random_state=1,max_iter=300)

X = train_mod[variaveis]
y = label
modelo.fit(X,y)

**Usando nos dados de teste:**

In [ ]:
X_prev = test_mod[variaveis]
pred = modelo.predict(X_prev)
predict_test = pd.Series(pred,name='Predict')
predict_test.value_counts()

In [ ]:
def sim_ou_nao(valor):
    if valor == 1:
        return 'Y'
    else:
        return 'N'
    
test_original['dep_delayed_15min'] = predict_test.map(sim_ou_nao)
test_original.head(10)

In [ ]:
test_original.to_csv("primeiro_modelo.csv",header=True)

In [ ]:
!head -n10 primeiro_modelo.csv